Carregando o dataset

In [1]:
import os
import pandas as pd
import numpy as np

dataset = pd.read_pickle(r'C:\Users\mathe\OneDrive\Área de Trabalho\SoccerIA\MathIA_v7\dataset_141cols_europeu.pkl')

nan_counts = dataset.isna().sum()
nan_tot = nan_counts.sum()
print(nan_tot)
print(dataset.shape)

0
(66840, 141)


Importando os dataframes de 2023

In [2]:
import os
import pandas as pd
import numpy as np

# Caminho para a pasta que contém os arquivos .xlsx de 2023
path = r'C:\Users\mathe\OneDrive\Área de Trabalho\SoccerIA\Planilhas2023'

# Dicionário para armazenar os dataframes
dataframes = {}
dfss = []
# Lista todos os arquivos na pasta
files = os.listdir(path)

# Filtra a lista de arquivos para incluir apenas os arquivos .xlsx
xlsx_files = [f for f in files if f.endswith('.xlsx')]

# Carrega cada arquivo .xlsx em um dataframe e armazena no dicionário
for file in xlsx_files:
    full_path = os.path.join(path, file)  # junta o caminho do diretório com o nome do arquivo
    dataframes[file] = pd.read_excel(full_path)  # lê o arquivo .xlsx do caminho completo

    if "Unnamed: 0" in dataframes[file].columns:
        dataframes[file].drop("Unnamed: 0", axis=1, inplace=True)

    if "match_report_url" in dataframes[file].columns:
        dataframes[file].drop("match_report_url", axis=1, inplace=True)    

    if "team1_yellow_cards" in dataframes[file].columns:
        dataframes[file].drop("team1_yellow_cards", axis=1, inplace=True)

    if "team2_yellow_cards" in dataframes[file].columns:
        dataframes[file].drop("team2_yellow_cards", axis=1, inplace=True)  
    dfss.append(dataframes[file])

    print(file)
    print(dataframes[file].shape)
    nan_counts = dataframes[file].isna().sum()
    print(nan_counts)


# Combine all the DataFrames into a single DataFrame
combined_df_2023 = pd.concat(dfss, ignore_index=True)


combined_df_2023['team1'] = combined_df_2023['team1'].str.lower()
combined_df_2023['team2'] = combined_df_2023['team2'].str.lower()
combined_df_2023.replace('', np.nan, inplace=True)
# Add a column to mark future matches
combined_df_2023['is_future_match'] = combined_df_2023['team1_goals'].isna() | combined_df_2023['team2_goals'].isna()
combined_df_2023['season'] = '2023'
# Replace empty string with NaN
combined_df_2023["team1_red_cards"].replace('', np.nan, inplace=True)
combined_df_2023["team2_red_cards"].replace('', np.nan, inplace=True)

# Replace NaN with 0
combined_df_2023["team1_red_cards"].fillna(0, inplace=True)
combined_df_2023["team2_red_cards"].fillna(0, inplace=True)

combined_df_2023['date'] = pd.to_datetime(combined_df_2023['date'], format='%Y-%m-%d', errors='coerce')
combined_df_2023.sort_values('date', inplace=True)

print(combined_df_2023.columns)
print(combined_df_2023.shape) 


dataframe2023_ALE_B.xlsx
(90, 19)
team1                     0
team2                     0
team1_goals              54
team2_goals              54
season                    0
championship              0
team1_shots_on_target    54
team1_shots_out          54
team2_shots_on_target    54
team2_shots_out          54
team1_red_cards          54
team2_red_cards          54
team1_fouls              54
team2_fouls              54
team1_corners            54
team2_corners            54
team1_total_shots        54
team2_total_shots        54
date                      0
dtype: int64
dataframe2023_BEL_A.xlsx
(77, 19)
team1                     0
team2                     0
team1_goals              40
team2_goals              40
season                    0
championship              0
team1_shots_on_target    40
team1_shots_out          40
team2_shots_on_target    40
team2_shots_out          40
team1_red_cards          40
team2_red_cards          40
team1_fouls              40
team2_fouls            

In [3]:
import pandas as pd
import numpy as np


combined_df_2023.sort_values('date', inplace=True)


combined_df_2023['team1_goals'] = pd.to_numeric(combined_df_2023['team1_goals'], errors='coerce')
combined_df_2023['team2_goals'] = pd.to_numeric(combined_df_2023['team2_goals'], errors='coerce')

# calculate goal differences
combined_df_2023['goal_diff_team1'] = combined_df_2023['team1_goals'] - combined_df_2023['team2_goals']
combined_df_2023['goal_diff_team2'] = combined_df_2023['team2_goals'] - combined_df_2023['team1_goals']

# calculate corners differences
combined_df_2023['corners_diff_team1'] = combined_df_2023['team1_corners'] - combined_df_2023['team2_corners']############# NEW
combined_df_2023['corners_diff_team2'] = combined_df_2023['team2_corners'] - combined_df_2023['team1_corners']############# NEW

# calculate big wins and losses
combined_df_2023['team1_big_win'] = np.where(combined_df_2023['goal_diff_team1'] >= 2, 1, 0)
combined_df_2023['team1_big_loss'] = np.where(combined_df_2023['goal_diff_team1'] <= -2, 1, 0)
combined_df_2023['team2_big_win'] = np.where(combined_df_2023['goal_diff_team2'] >= 2, 1, 0)
combined_df_2023['team2_big_loss'] = np.where(combined_df_2023['goal_diff_team2'] <= -2, 1, 0)

# calculate AH-2.5 win and losses
combined_df_2023['team1_ah-2.5_win'] = np.where(combined_df_2023['corners_diff_team1'] >= 3, 1, 0)############# NEW
combined_df_2023['team1_ah-2.5_loss'] = np.where(combined_df_2023['corners_diff_team1'] <= 2, 1, 0)############# NEW
combined_df_2023['team2_ah-2.5_win'] = np.where(combined_df_2023['corners_diff_team2'] >= 3, 1, 0)############# NEW
combined_df_2023['team2_ah-2.5_loss'] = np.where(combined_df_2023['corners_diff_team2'] <= 2, 1, 0)############# NEW


# calculate AH+2.5 win and losses
combined_df_2023['team1_ah+2.5_win'] = np.where(combined_df_2023['corners_diff_team1'] >= -2, 1, 0)############# NEW
combined_df_2023['team1_ah+2.5_loss'] = np.where(combined_df_2023['corners_diff_team1'] <= -3, 1, 0)############# NEW
combined_df_2023['team2_ah+2.5_win'] = np.where(combined_df_2023['corners_diff_team2'] >= -2, 1, 0)############# NEW
combined_df_2023['team2_ah+2.5_loss'] = np.where(combined_df_2023['corners_diff_team2'] <= -3, 1, 0)############# NEW


# calculate over4.5 win and losses
combined_df_2023['team1_over4.5'] = np.where(combined_df_2023['team1_corners'] >= 5, 1, 0)############# NEW
combined_df_2023['team1_under4.5'] = np.where(combined_df_2023['team1_corners'] <= 4, 1, 0)############# NEW
combined_df_2023['team2_over4.5'] = np.where(combined_df_2023['team2_corners'] >= 5, 1, 0)############# NEW
combined_df_2023['team2_under4.5'] = np.where(combined_df_2023['team2_corners'] <= 4, 1, 0)############# NEW

# calculate over3.5 win and losses
combined_df_2023['team1_over3.5'] = np.where(combined_df_2023['team1_corners'] >= 4, 1, 0)############# NEW
combined_df_2023['team1_under3.5'] = np.where(combined_df_2023['team1_corners'] <= 3, 1, 0)############# NEW
combined_df_2023['team2_over3.5'] = np.where(combined_df_2023['team2_corners'] >= 4, 1, 0)############# NEW
combined_df_2023['team2_under3.5'] = np.where(combined_df_2023['team2_corners'] <= 3, 1, 0)############# NEW

# calculate over6.5 win and losses
combined_df_2023['team1_over6.5'] = np.where(combined_df_2023['team1_corners'] >= 7, 1, 0)############# NEW
combined_df_2023['team1_under6.5'] = np.where(combined_df_2023['team1_corners'] <= 6, 1, 0)############# NEW
combined_df_2023['team2_over6.5'] = np.where(combined_df_2023['team2_corners'] >= 7, 1, 0)############# NEW
combined_df_2023['team2_under6.5'] = np.where(combined_df_2023['team2_corners'] <= 6, 1, 0)############# NEW


# Initialize these columns with 0
combined_df_2023['team1_big_wins_last5'] = 0
combined_df_2023['team1_big_losses_last5'] = 0
combined_df_2023['team2_big_wins_last5'] = 0
combined_df_2023['team2_big_losses_last5'] = 0

combined_df_2023['team1_ah-2.5_wins_last5'] = 0############# NEW
combined_df_2023['team1_ah-2.5_losses_last5'] = 0############# NEW
combined_df_2023['team2_ah-2.5_wins_last5'] = 0############# NEW
combined_df_2023['team2_ah-2.5_losses_last5'] = 0############# NEW

combined_df_2023['team1_ah+2.5_wins_last5'] = 0############# NEW
combined_df_2023['team1_ah+2.5_losses_last5'] = 0############# NEW
combined_df_2023['team2_ah+2.5_wins_last5'] = 0############# NEW
combined_df_2023['team2_ah+2.5_losses_last5'] = 0############# NEW

combined_df_2023['team1_over3.5_last5'] = 0############# NEW
combined_df_2023['team1_under3.5_last5'] = 0############# NEW
combined_df_2023['team2_over3.5_last5'] = 0############# NEW
combined_df_2023['team2_under3.5_last5'] = 0############# NEW

combined_df_2023['team1_over4.5_last5'] = 0############# NEW
combined_df_2023['team1_under4.5_last5'] = 0############# NEW
combined_df_2023['team2_over4.5_last5'] = 0############# NEW
combined_df_2023['team2_under4.5_last5'] = 0############# NEW

combined_df_2023['team1_over6.5_last5'] = 0############# NEW
combined_df_2023['team1_under6.5_last5'] = 0############# NEW
combined_df_2023['team2_over6.5_last5'] = 0############# NEW
combined_df_2023['team2_under6.5_last5'] = 0############# NEW


new_cols = ['avg_scr_lasts3_1_home', 'avg_scr_lasts5_1_home', 'avg_scr_lasts3_1_away',
            'avg_scr_lasts5_1_away', 'avg_conc_lasts3_1_home', 'avg_conc_lasts5_1_home',
            'avg_conc_lasts3_1_away', 'avg_conc_lasts5_1_away', 'avg_scr_lasts3_2_home',
            'avg_scr_lasts5_2_home', 'avg_scr_lasts3_2_away', 'avg_scr_lasts5_2_away',
            'avg_conc_lasts3_2_home', 'avg_conc_lasts5_2_home', 'avg_conc_lasts3_2_away',
            'avg_conc_lasts5_2_away','team1_big_wins_last5', 'team1_big_losses_last5', 
            'team2_big_wins_last5', 'team2_big_losses_last5',
            #abaixo vai ser baseado em finalizações
            'avg_total_shots_lasts5_1_home','avg_total_shots_lasts5_1_away','avg_total_shots_lasts5_2_home',
            'avg_total_shots_lasts5_2_away', 'avg_otarget_shots_lasts5_1_home','avg_otarget_shots_lasts5_1_away',
            'avg_otarget_shots_lasts5_2_home','avg_otarget_shots_lasts5_2_away','avg_out_shots_lasts5_1_home',
            'avg_out_shots_lasts5_1_away','avg_out_shots_lasts5_2_home','avg_out_shots_lasts5_2_away',
            'avg_conc_total_shots_lasts5_1_home','avg_conc_total_shots_lasts5_1_away',
            'avg_conc_total_shots_lasts5_2_home','avg_conc_total_shots_lasts5_2_away',
            #abaixo vai ser baseado em corners
            'avg_corners_lasts5_1_home','avg_corners_lasts5_1_away', 
            'avg_corners_conc_lasts5_1_home','avg_corners_conc_lasts5_1_away',
            'avg_corners_lasts5_2_home','avg_corners_lasts5_2_away', 
            'avg_corners_conc_lasts5_2_home', 'avg_corners_conc_lasts5_2_away',
            #abaixo vai ser baseado em fouls
            'avg_fouls_lasts5_1_home','avg_fouls_lasts5_1_away', 
            'avg_fouls_conc_lasts5_1_home', 'avg_fouls_conc_lasts5_1_away',
            'avg_fouls_lasts5_2_home','avg_fouls_lasts5_2_away', 
            'avg_fouls_conc_lasts5_2_home', 'avg_fouls_conc_lasts5_2_away',
            #novas colunas da v7
            'team1_ah-2.5_wins_last5', 'team1_ah-2.5_losses_last5','team2_ah-2.5_wins_last5','team2_ah-2.5_losses_last5',
            'team1_ah+2.5_wins_last5','team1_ah+2.5_losses_last5','team2_ah+2.5_wins_last5','team2_ah+2.5_losses_last5',
            'team1_over3.5_last5','team1_under3.5_last5','team2_over3.5_last5','team2_under3.5_last5',
            'team1_over4.5_last5','team1_under4.5_last5','team2_over4.5_last5','team2_under4.5_last5',
            'team1_over6.5_last5','team1_under6.5_last5','team2_over6.5_last5','team2_under6.5_last5'
            ]

# Create a dictionary with keys as column names and values as np.nan
new_cols_dict = {col: np.nan for col in new_cols}

# Add new columns to the DataFrame
combined_df_2023 = combined_df_2023.assign(**new_cols_dict)


# Iterate over each row in the DataFrame
for i, row in combined_df_2023.iterrows():
    # For each team, get their past home and away matches before the current date
    team1_matches = combined_df_2023[((combined_df_2023['team1'] == row['team1']) | (combined_df_2023['team2'] == row['team1'])) & (combined_df_2023['date'] < row['date']) & (combined_df_2023['season'] == row['season'])].sort_values(by='date')
    team2_matches = combined_df_2023[((combined_df_2023['team1'] == row['team2']) | (combined_df_2023['team2'] == row['team2'])) & (combined_df_2023['date'] < row['date']) & (combined_df_2023['season'] == row['season'])].sort_values(by='date')

    # For each team, calculate stats for last 5 matches
    if not team1_matches.empty:
        team1_matches['big_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_big_win'], team1_matches['team2_big_win'])
        team1_matches['big_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_big_loss'], team1_matches['team2_big_loss'])
        combined_df_2023.at[i, 'team1_big_wins_last5'] = team1_matches['big_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_big_losses_last5'] = team1_matches['big_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para ah-2.5 para a equipe 1
        team1_matches['ah-2.5_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah-2.5_win'], team1_matches['team2_ah-2.5_win'])
        team1_matches['ah-2.5_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah-2.5_loss'], team1_matches['team2_ah-2.5_loss'])
        combined_df_2023.at[i, 'team1_ah-2.5_wins_last5'] = team1_matches['ah-2.5_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_ah-2.5_losses_last5'] = team1_matches['ah-2.5_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para ah+2.5 para a equipe 1
        team1_matches['ah+2.5_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah+2.5_win'], team1_matches['team2_ah+2.5_win'])
        team1_matches['ah+2.5_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah+2.5_loss'], team1_matches['team2_ah+2.5_loss'])
        combined_df_2023.at[i, 'team1_ah+2.5_wins_last5'] = team1_matches['ah+2.5_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_ah+2.5_losses_last5'] = team1_matches['ah+2.5_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over3.5 para a equipe 1
        team1_matches['over3.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over3.5'], team1_matches['team2_over3.5'])
        team1_matches['under3.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under3.5'], team1_matches['team2_under3.5'])
        combined_df_2023.at[i, 'team1_over3.5_last5'] = team1_matches['over3.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under3.5_last5'] = team1_matches['under3.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over4.5 para a equipe 1
        team1_matches['over4.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over4.5'], team1_matches['team2_over4.5'])
        team1_matches['under4.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under4.5'], team1_matches['team2_under4.5'])
        combined_df_2023.at[i, 'team1_over4.5_last5'] = team1_matches['over4.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under4.5_last5'] = team1_matches['under4.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over6.5 para a equipe 1
        team1_matches['over6.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over6.5'], team1_matches['team2_over6.5'])
        team1_matches['under6.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under6.5'], team1_matches['team2_under6.5'])
        combined_df_2023.at[i, 'team1_over6.5_last5'] = team1_matches['over6.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under6.5_last5'] = team1_matches['under6.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan



    if not team2_matches.empty:
        team2_matches['big_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_big_win'], team2_matches['team2_big_win'])
        team2_matches['big_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_big_loss'], team2_matches['team2_big_loss'])
        combined_df_2023.at[i, 'team2_big_wins_last5'] = team2_matches['big_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_big_losses_last5'] = team2_matches['big_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para ah-2.5 para a equipe 2
        team2_matches['ah-2.5_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah-2.5_win'], team2_matches['team2_ah-2.5_win'])
        team2_matches['ah-2.5_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah-2.5_loss'], team2_matches['team2_ah-2.5_loss'])
        combined_df_2023.at[i, 'team2_ah-2.5_wins_last5'] = team2_matches['ah-2.5_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_ah-2.5_losses_last5'] = team2_matches['ah-2.5_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para ah+2.5 para a equipe 2
        team2_matches['ah+2.5_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah+2.5_win'], team2_matches['team2_ah+2.5_win'])
        team2_matches['ah+2.5_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah+2.5_loss'], team2_matches['team2_ah+2.5_loss'])
        combined_df_2023.at[i, 'team2_ah+2.5_wins_last5'] = team2_matches['ah+2.5_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_ah+2.5_losses_last5'] = team2_matches['ah+2.5_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over3.5  para a equipe 2
        team2_matches['over3.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over3.5'], team2_matches['team2_over3.5'])
        team2_matches['under3.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under3.5'], team2_matches['team2_under3.5'])
        combined_df_2023.at[i, 'team2_over3.5_last5'] = team2_matches['over3.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under3.5_last5'] = team2_matches['under3.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over4.5  para a equipe 2
        team2_matches['over4.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over4.5'], team2_matches['team2_over4.5'])
        team2_matches['under4.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under4.5'], team2_matches['team2_under4.5'])
        combined_df_2023.at[i, 'team2_over4.5_last5'] = team2_matches['over4.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under4.5_last5'] = team2_matches['under4.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over6.5  para a equipe 2
        team2_matches['over6.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over6.5'], team2_matches['team2_over6.5'])
        team2_matches['under6.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under6.5'], team2_matches['team2_under6.5'])
        combined_df_2023.at[i, 'team2_over6.5_last5'] = team2_matches['over6.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under6.5_last5'] = team2_matches['under6.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan



    
for i, row in combined_df_2023.iterrows():
    team1_home = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team1'] == row['team1']) & (combined_df_2023['season'] == row['season'])]
    team1_away = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team2'] == row['team1']) & (combined_df_2023['season'] == row['season'])]
    
    team2_home = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team1'] == row['team2']) & (combined_df_2023['season'] == row['season'])]
    team2_away = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team2'] == row['team2']) & (combined_df_2023['season'] == row['season'])]

    if not team1_home.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_1_home'] = team1_home['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_1_home'] = team1_home['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_1_home'] = team1_home['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_home['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_1_home'] = team1_home['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_1_home'] = team1_home['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_1_home'] = team1_home['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_1_home'] = team1_home['team1_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_1_home'] = team1_home['team1_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_shots_out'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_1_home'] = team1_home['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_1_home'] = team1_home['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_1_home'] = team1_home['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_1_home'] = team1_home['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_fouls'].isna().any() else np.nan

    if not team1_away.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_1_away'] = team1_away['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_1_away'] = team1_away['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_1_away'] = team1_away['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_away['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_1_away'] = team1_away['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_1_away'] = team1_away['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_1_away'] = team1_away['team2_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_1_away'] = team1_away['team2_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_1_away'] = team1_away['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_1_away'] = team1_away['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_1_away'] = team1_away['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_1_away'] = team1_away['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_1_away'] = team1_away['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_fouls'].isna().any() else np.nan

    if not team2_home.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_2_home'] = team2_home['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_2_home'] = team2_home['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_2_home'] = team2_home['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_home['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_2_home'] = team2_home['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_2_home'] = team2_home['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_2_home'] = team2_home['team1_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_2_home'] = team2_home['team1_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_2_home'] = team2_home['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_2_home'] = team2_home['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_2_home'] = team2_home['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_2_home'] = team2_home['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_2_home'] = team2_home['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_fouls'].isna().any() else np.nan


    if not team2_away.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_2_away'] = team2_away['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_2_away'] = team2_away['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_2_away'] = team2_away['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_away['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_2_away'] = team2_away['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_2_away'] = team2_away['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_2_away'] = team2_away['team2_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_2_away'] = team2_away['team2_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_2_away'] = team2_away['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_2_away'] = team2_away['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_2_away'] = team2_away['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_2_away'] = team2_away['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_2_away'] = team2_away['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_fouls'].isna().any() else np.nan
        




combined_df_2023.shape        

(1076, 120)

In [4]:
def get_result(row):
    if row['team1_goals'] > row['team2_goals']:
        return pd.Series([3, 0])
    elif row['team1_goals'] < row['team2_goals']:
        return pd.Series([0, 3])
    else:
        return pd.Series([1, 1])

combined_df_2023[['result_team1', 'result_team2']] = combined_df_2023.apply(get_result, axis=1)

def get_streak(df, result_col, results):
    result_series = df[result_col].apply(lambda x: 1 if x in results else 0)
    result_series = result_series * (result_series.groupby((result_series != result_series.shift()).cumsum()).cumcount() + 1)
    return result_series

# Create a dictionary to hold individual team dataframes
team_df_dict = {}

def get_individual_team_df(df, team_name): #teoricamente aqui deveria ser corners, mas o erro se apresentou menor assim:
    if team_name in team_df_dict:
        return team_df_dict[team_name]
        
    team_games = df[(df['team1'] == team_name) | (df['team2'] == team_name)].copy()
    team_games['team_is_team1'] = team_games['team1'] == team_name
    team_games['team_result'] = np.where(team_games['team_is_team1'], team_games['result_team1'], team_games['result_team2'])
    team_games['team_goals'] = np.where(team_games['team_is_team1'], team_games['team1_goals'], team_games['team2_goals'])
    team_games['team_redcards'] = np.where(team_games['team_is_team1'], team_games['team1_red_cards'], team_games['team2_red_cards'])

    team_games.sort_values('date', inplace=True)
    team_games['days_since_last_game'] = team_games['date'].diff().dt.days

    team_df_dict[team_name] = team_games
    return team_games

def get_team_stats(row, df):
    team1_games = get_individual_team_df(df, row['team1'])
    team2_games = get_individual_team_df(df, row['team2'])

    # Filter to include only games that occurred before the current game
    team1_games = team1_games[team1_games['date'] < row['date']]
    team2_games = team2_games[team2_games['date'] < row['date']]

    stats = {}

    if not team1_games.empty:
        stats['team1_winning_streak'] = get_streak(team1_games, 'team_result', [3]).iloc[-1]
        stats['team1_undefeated_streak'] = get_streak(team1_games, 'team_result', [1, 3]).iloc[-1]
        stats['team1_losing_streak'] = get_streak(team1_games, 'team_result', [0]).iloc[-1]
        stats['team1_without_winning_streak'] = get_streak(team1_games, 'team_result', [0, 1]).iloc[-1]
        stats['avg_points_lasts5_1'] = team1_games.tail(5)['team_result'].mean()
        stats['team1_strength'] = team1_games['team_goals'].sum() / (team1_games['team1_goals'].sum() + team1_games['team2_goals'].sum() + 0.01)
        stats['championship_points_1'] = team1_games['team_result'].sum() / len(team1_games)
        rested_4_or_more_days_1 = team1_games.tail(1)['days_since_last_game'].values[0] >= 4
        stats['rested_4_days_or_more_1'] = 1 if rested_4_or_more_days_1 else -1

    if not team2_games.empty:
        stats['team2_winning_streak'] = get_streak(team2_games, 'team_result', [3]).iloc[-1]
        stats['team2_undefeated_streak'] = get_streak(team2_games, 'team_result', [1, 3]).iloc[-1]
        stats['team2_losing_streak'] = get_streak(team2_games, 'team_result', [0]).iloc[-1]
        stats['team2_without_winning_streak'] = get_streak(team2_games, 'team_result', [0, 1]).iloc[-1]
        stats['avg_points_lasts5_2'] = team2_games.tail(5)['team_result'].mean()
        stats['team2_strength'] = team2_games['team_goals'].sum() / (team2_games['team1_goals'].sum() + team2_games['team2_goals'].sum() + 0.01)
        stats['championship_points_2'] = team2_games['team_result'].sum() / len(team2_games)
        rested_4_or_more_days_2 = team2_games.tail(1)['days_since_last_game'].values[0] >= 4
        stats['rested_4_days_or_more_2'] = 1 if rested_4_or_more_days_2 else -1

    return pd.Series(stats)

combined_df_2023 = pd.concat([combined_df_2023, combined_df_2023.apply(lambda row: get_team_stats(row, combined_df_2023), axis=1)], axis=1)

# Now, calculate the number of suspended players for the next match for each team.
for team_name in team_df_dict.keys():
    team_df = team_df_dict[team_name].copy()
    team_df['next_match_suspended_players'] = team_df['team_redcards'].shift()

    # Assign the suspended players back to the combined_df_2023.
    team1_mask = combined_df_2023['team1'] == team_name
    team2_mask = combined_df_2023['team2'] == team_name
    combined_df_2023.loc[team1_mask, 'team1_suspended_players'] = team_df.loc[team1_mask, 'next_match_suspended_players']
    combined_df_2023.loc[team2_mask, 'team2_suspended_players'] = team_df.loc[team2_mask, 'next_match_suspended_players']

combined_df_2023.shape    


(1076, 140)

In [5]:
list(combined_df_2023.columns)

['team1',
 'team2',
 'team1_goals',
 'team2_goals',
 'season',
 'championship',
 'team1_shots_on_target',
 'team1_shots_out',
 'team2_shots_on_target',
 'team2_shots_out',
 'team1_red_cards',
 'team2_red_cards',
 'team1_fouls',
 'team2_fouls',
 'team1_corners',
 'team2_corners',
 'team1_total_shots',
 'team2_total_shots',
 'date',
 'is_future_match',
 'goal_diff_team1',
 'goal_diff_team2',
 'corners_diff_team1',
 'corners_diff_team2',
 'team1_big_win',
 'team1_big_loss',
 'team2_big_win',
 'team2_big_loss',
 'team1_ah-2.5_win',
 'team1_ah-2.5_loss',
 'team2_ah-2.5_win',
 'team2_ah-2.5_loss',
 'team1_ah+2.5_win',
 'team1_ah+2.5_loss',
 'team2_ah+2.5_win',
 'team2_ah+2.5_loss',
 'team1_over4.5',
 'team1_under4.5',
 'team2_over4.5',
 'team2_under4.5',
 'team1_over3.5',
 'team1_under3.5',
 'team2_over3.5',
 'team2_under3.5',
 'team1_over6.5',
 'team1_under6.5',
 'team2_over6.5',
 'team2_under6.5',
 'team1_big_wins_last5',
 'team1_big_losses_last5',
 'team2_big_wins_last5',
 'team2_big_loss

In [6]:
columns_to_drop = [
 #'team1_goals',
 #'team2_goals',
 'team1_shots_on_target',
 'team1_shots_out',
 'team2_shots_on_target',
 'team2_shots_out',
 'team1_red_cards',
 'team2_red_cards',
 'team1_fouls',
 'team2_fouls',
 'team1_corners',
 'team2_corners',
 'team1_total_shots',
 'team2_total_shots',
 'date',
 'is_future_match',
 'goal_diff_team1',
 'goal_diff_team2',
 'corners_diff_team1',
 'corners_diff_team2',
 'team1_big_win',
 'team1_big_loss',
 'team2_big_win',
 'team2_big_loss',
 'team1_ah-2.5_win',
 'team1_ah-2.5_loss',
 'team2_ah-2.5_win',
 'team2_ah-2.5_loss',
 'team1_ah+2.5_win',
 'team1_ah+2.5_loss',
 'team2_ah+2.5_win',
 'team2_ah+2.5_loss',
 'team1_over4.5',
 'team1_under4.5',
 'team2_over4.5',
 'team2_under4.5',
 'team1_over3.5',
 'team1_under3.5',
 'team2_over3.5',
 'team2_under3.5',
 'team1_over6.5',
 'team1_under6.5',
 'team2_over6.5',
 'team2_under6.5',
 'result_team1',
 'result_team2']

Criando future Match baseado na data de hoje

In [7]:
dataset2 = combined_df_2023.copy()
print(combined_df_2023.shape)
# filter the DataFrame
future_matches = dataset2[dataset2['is_future_match'] == True].copy()
future_matches.sort_values(by='date')
print(future_matches.shape)

from datetime import datetime, timedelta

# Get yesterday's date
yesterday = datetime.now() - timedelta(days=1)

# Filter the DataFrame to include only rows with dates greater than yesterday
#future_matches = future_matches[future_matches['date'] > yesterday]

# Substituir np.nan por 0
future_matches['team1_goals'].fillna(0, inplace=True)
future_matches['team2_goals'].fillna(0, inplace=True)

# Liste todas as colunas que você deseja verificar
columns_to_check= [col for col in future_matches.columns if col not in columns_to_drop]

# Drop as linhas com 'np.nan' nas colunas especificadas
future_matches = future_matches.dropna(subset=columns_to_check)
future_matches_calculado = future_matches.drop(columns_to_drop,axis=1)
future_matches_calculado = future_matches_calculado.drop('season',axis=1)


"""# Contando os valores NaN em cada coluna
nan_counts = future_matches.isna().sum()

# Transformando em uma lista de pares (nome da coluna, contagem de np.nan)
nan_list = list(nan_counts.items())

# Percorrendo a lista e imprimindo cada valor individualmente com o nome da coluna
for col_name, nan_count in nan_list:
    print(f'{col_name}: {nan_count}')"""
    
print(future_matches_calculado.shape)

(1076, 140)
(445, 140)
(43, 95)


In [8]:
champ_uniques = future_matches_calculado['championship'].unique()
print(champ_uniques)

['BEL A' 'SUE A' 'BRA A' 'ING C' 'ING D']


In [9]:
future_matches_calculado.head()

,team1,team2,team1_goals,team2_goals,championship,team1_big_wins_last5,team1_big_losses_last5,team2_big_wins_last5,team2_big_losses_last5,team1_ah-2.5_wins_last5,...,team2_winning_streak,team2_undefeated_streak,team2_losing_streak,team2_without_winning_streak,avg_points_lasts5_2,team2_strength,championship_points_2,rested_4_days_or_more_2,team1_suspended_players,team2_suspended_players
127,oud-heverlee leuven,kv kortrijk,0.0,0.0,BEL A,0.0,2.0,0.0,2.0,3.0,...,0.0,1.0,0.0,5.0,0.2,0.222099,0.200000,1.0,0.0,0.0
1031,ifk varnamo,elfsborg,0.0,0.0,SUE A,2.0,1.0,1.0,1.0,1.0,...,2.0,2.0,0.0,0.0,1.8,0.730043,2.285714,1.0,0.0,0.0
372,athletico-pr,atlético mineiro,0.0,0.0,BRA A,1.0,0.0,2.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.8,0.571293,1.428571,1.0,1.0,0.0
373,goiás,internacional,0.0,0.0,BRA A,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,9.0,0.4,0.414533,1.190476,1.0,0.0,0.0
374,botafogo,flamengo,0.0,0.0,BRA A,3.0,0.0,0.0,1.0,2.0,...,0.0,3.0,0.0,1.0,1.6,0.566572,1.714286,1.0,0.0,0.0


In [10]:
#CORTAR AS LINHAS COM MATCHES FUTUROS AQUI

# Replace empty strings with NaN
print(dataset2.shape)
dataset2.replace('', np.nan, inplace=True)
print(f"A quantidade de np.nan em linhas eram {dataset2.isna().sum().sum()}")
# Remove rows that contain any missing values
dataset2.dropna(inplace=True)

import numpy as np

counter = 0  # Initialize counter
for index, row in dataset2.iterrows():
    if row.isnull().any() or row.eq('').any():
        print(f"Row {index} contains 'NaN' or an empty value.")
        counter += 1  # Increase counter if condition is met

print(f"Total number of rows with 'NaN' or an empty value: {counter}")
dataset2.shape


(1076, 140)
A quantidade de np.nan em linhas eram 44744
Total number of rows with 'NaN' or an empty value: 0


(302, 140)

Concatenando os 2

In [11]:
datatotal = pd.concat([dataset, dataset2], ignore_index=True)#mudei o 'dataset' por combined_df_13c_new

datatotal.sort_values(by='date', inplace=True)

if 'team1_yellow_cards' in datatotal.columns:
    datatotal = datatotal.drop(['team1_yellow_cards'], axis=1)

if 'team2_yellow_cards' in datatotal.columns:
    datatotal = datatotal.drop(['team2_yellow_cards'], axis=1)

# Substituir valores maiores que 15 por 15 na coluna 'team1_corners'
datatotal.loc[datatotal['team1_corners'] > 15, 'team1_corners'] = 15

# Substituir valores maiores que 15 por 15 na coluna 'team2_corners'
datatotal.loc[datatotal['team2_corners'] > 15, 'team2_corners'] = 15    

datatotal

,championship,date,team1,team2,team1_goals,team2_goals,team1_total_shots,team2_total_shots,team1_shots_on_target,team2_shots_on_target,...,team1_winning_streak,team1_without_winning_streak,team2_losing_streak,team2_strength,team2_undefeated_streak,team2_winning_streak,team2_without_winning_streak,team1_suspended_players,team2_suspended_players,is_future_match
0,E0,2002-09-14,Charlton,Arsenal,0.0,3.0,9.0,10.0,3.0,8.0,...,0.0,1.0,0.0,0.666297,5.0,1.0,0.0,0.0,0.0,NaN
1,E0,2002-09-14,Everton,Middlesbrough,2.0,1.0,13.0,10.0,8.0,5.0,...,0.0,3.0,0.0,0.665927,1.0,1.0,0.0,0.0,0.0,NaN
2,E0,2002-09-14,Leeds,Man United,1.0,0.0,8.0,6.0,2.0,5.0,...,1.0,0.0,1.0,0.554939,0.0,0.0,1.0,0.0,0.0,NaN
3,E0,2002-09-14,West Brom,Southampton,1.0,0.0,11.0,10.0,7.0,5.0,...,2.0,0.0,0.0,0.332963,1.0,1.0,0.0,0.0,0.0,NaN
4,E0,2002-09-15,Man City,Blackburn,2.0,2.0,15.0,12.0,7.0,8.0,...,0.0,1.0,2.0,0.454133,0.0,0.0,3.0,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67124,ING D,2023-09-02,harrogate town fc,barrow,0.0,1.0,10.0,8.0,6.0,3.0,...,1.0,0.0,0.0,0.544959,1.0,0.0,3.0,0.0,0.0,False
67123,ING D,2023-09-02,morecambe,salford city,1.0,0.0,9.0,4.0,5.0,2.0,...,0.0,1.0,1.0,0.499688,0.0,0.0,1.0,1.0,0.0,False
67140,ING C,2023-09-02,bolton wanderers fc,derby county,2.0,1.0,10.0,5.0,6.0,1.0,...,0.0,2.0,0.0,0.624610,2.0,2.0,0.0,0.0,0.0,False
67131,ING D,2023-09-02,newport county afc,afc wimbledon,2.0,2.0,8.0,11.0,4.0,4.0,...,2.0,0.0,0.0,0.776915,5.0,0.0,1.0,0.0,0.0,False


In [12]:
champ_uniques = datatotal['championship'].unique()
print(champ_uniques)

['E0' 'SC0' 'E3' 'E2' 'E1' 'I1' 'SP1' 'D1' 'F1' 'D2' 'P1' 'SP2' 'T1' 'I2'
 'N1' 'F2' 'B1' 'G1' 'SUE A' 'BRA A' 'ING C' 'ING D']


Converta Categorias em IDs Numéricos

In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Suponho que 'datatotal' e 'future_matches' já estejam definidos e tenham as mesmas colunas relevantes

# Treinar o LabelEncoder com 'datatotal' e 'future_matches'
le_teams = LabelEncoder().fit(pd.concat([datatotal['team1'], datatotal['team2'], future_matches_calculado['team1'], future_matches_calculado['team2']]).astype(str))
le_champ = LabelEncoder().fit(pd.concat([datatotal['championship'], future_matches_calculado['championship']]).astype(str))

# Aplicar o LabelEncoder a 'datatotal'
datatotal['team1'] = le_teams.transform(datatotal['team1'].astype(str))
datatotal['team2'] = le_teams.transform(datatotal['team2'].astype(str))
datatotal['championship'] = le_champ.transform(datatotal['championship'].astype(str))

# Agora, aplicar o mesmo LabelEncoder a 'future_matches'
future_matches_calculado['team1'] = le_teams.transform(future_matches_calculado['team1'].astype(str))
future_matches_calculado['team2'] = le_teams.transform(future_matches_calculado['team2'].astype(str))
future_matches_calculado['championship'] = le_champ.transform(future_matches_calculado['championship'].astype(str))



In [14]:
future_matches_calculado.head()

,team1,team2,team1_goals,team2_goals,championship,team1_big_wins_last5,team1_big_losses_last5,team2_big_wins_last5,team2_big_losses_last5,team1_ah-2.5_wins_last5,...,team2_winning_streak,team2_undefeated_streak,team2_losing_streak,team2_without_winning_streak,avg_points_lasts5_2,team2_strength,championship_points_2,rested_4_days_or_more_2,team1_suspended_players,team2_suspended_players
127,533,519,0.0,0.0,1,0.0,2.0,0.0,2.0,3.0,...,0.0,1.0,0.0,5.0,0.2,0.222099,0.200000,1.0,0.0,0.0
1031,515,499,0.0,0.0,21,2.0,1.0,1.0,1.0,1.0,...,2.0,2.0,0.0,0.0,1.8,0.730043,2.285714,1.0,0.0,0.0
372,471,472,0.0,0.0,2,1.0,0.0,2.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.8,0.571293,1.428571,1.0,1.0,0.0
373,508,517,0.0,0.0,2,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,9.0,0.4,0.414533,1.190476,1.0,0.0,0.0
374,479,502,0.0,0.0,2,3.0,0.0,0.0,1.0,2.0,...,0.0,3.0,0.0,1.0,1.6,0.566572,1.714286,1.0,0.0,0.0


In [15]:
n_teams = len(le_teams.classes_)
n_champ = len(le_champ.classes_)
print(n_teams)
print(n_champ)

555
23


In [16]:
datatotal['season'] = datatotal['season'].astype('float64')


Separando X e Y

In [17]:
X = datatotal.drop(columns_to_drop, axis=1)
y1 = datatotal['team1_corners']
y2 = datatotal['team2_corners']
X.shape

(67142, 96)

In [18]:
list(X.dtypes)

[dtype('int32'),
 dtype('int32'),
 dtype('int32'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),

In [19]:
list(X.columns)

['championship',
 'team1',
 'team2',
 'team1_goals',
 'team2_goals',
 'season',
 'team1_big_wins_last5',
 'team1_big_losses_last5',
 'team2_big_wins_last5',
 'team2_big_losses_last5',
 'team1_ah-2.5_wins_last5',
 'team1_ah-2.5_losses_last5',
 'team2_ah-2.5_wins_last5',
 'team2_ah-2.5_losses_last5',
 'team1_ah+2.5_wins_last5',
 'team1_ah+2.5_losses_last5',
 'team2_ah+2.5_wins_last5',
 'team2_ah+2.5_losses_last5',
 'team1_over3.5_last5',
 'team1_under3.5_last5',
 'team2_over3.5_last5',
 'team2_under3.5_last5',
 'team1_over4.5_last5',
 'team1_under4.5_last5',
 'team2_over4.5_last5',
 'team2_under4.5_last5',
 'team1_over6.5_last5',
 'team1_under6.5_last5',
 'team2_over6.5_last5',
 'team2_under6.5_last5',
 'avg_scr_lasts3_1_home',
 'avg_scr_lasts5_1_home',
 'avg_scr_lasts3_1_away',
 'avg_scr_lasts5_1_away',
 'avg_conc_lasts3_1_home',
 'avg_conc_lasts5_1_home',
 'avg_conc_lasts3_1_away',
 'avg_conc_lasts5_1_away',
 'avg_scr_lasts3_2_home',
 'avg_scr_lasts5_2_home',
 'avg_scr_lasts3_2_away',


In [20]:
y1_uniques = datatotal['team1_corners'].unique()
print(y1_uniques)

[ 1. 10.  4.  0. 15.  9.  8.  5.  6.  7.  3. 12. 11. 14.  2. 13.]


In [21]:
# Contar ocorrências para 'y1'
y1_counts = y1.value_counts().sort_index().reindex(range(0, 31), fill_value=0)
print("Contagem de ocorrências para y1:")
print(y1_counts)

# Contar ocorrências para 'y2'
y2_counts = y2.value_counts().sort_index().reindex(range(0, 31), fill_value=0)
print("\nContagem de ocorrências para y2:")
print(y2_counts)


Contagem de ocorrências para y1:
team1_corners
0      695
1     2435
2     5027
3     7578
4     9210
5     9504
6     8627
7     7240
8     5578
9     3952
10    2777
11    1818
12    1112
13     711
14     416
15     462
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
Name: count, dtype: int64

Contagem de ocorrências para y2:
team2_corners
0      1556
1      4940
2      8154
3     10281
4     10644
5      9473
6      7390
7      5630
8      3583
9      2336
10     1369
11      847
12      415
13      283
14      137
15      104
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
30        0
Name: count, dtype: int64


Separando o treino e o teste e a normalização

Algumas variáveis abaixo estão sendo criadas de forma idêntica, apenas para ter significado semântico em relação 

In [25]:
from sklearn.preprocessing import StandardScaler

# Dividindo os dados com base na coluna 'season'
X_train = X[X['season'] < 2022].drop(['team1', 'team2', 'championship', 'season'], axis=1)
X_test = X[X['season'] >= 2022].drop(['team1', 'team2', 'championship', 'season'], axis=1)
y_train1 = y1[X['season'] < 2022]
y_test1 = y1[X['season'] >= 2022]
y_train2 = y2[X['season'] < 2022]
y_test2 = y2[X['season'] >= 2022]

# Escalando apenas as colunas que você quer (ajuste isso conforme suas necessidades)
cols_to_scale = [col for col in future_matches_calculado.columns if col not in ['team1', 'team2', 'championship']]

# Ajustar o escalonador com base no conjunto de treinamento
scaler = StandardScaler().fit(X_train[cols_to_scale])

# Reordenar as colunas para corresponder à ordem usada para ajustar o escalonador
X_train = X_train[cols_to_scale]
X_test = X_test[cols_to_scale]

# Aplicar o escalonamento
future_matches_calculado_scaled = future_matches_calculado.copy()
future_matches_calculado_scaled[cols_to_scale] = scaler.transform(future_matches_calculado[cols_to_scale])

# Transformar os conjuntos de treinamento e teste
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Preparar as colunas para o embedding
X_train_embed = X[X['season'] < 2022][['team1', 'team2', 'championship']]
X_test_embed = X[X['season'] >= 2022][['team1', 'team2', 'championship']]


In [ ]:
list(X_train1.columns)

In [27]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train1.shape)
print(y_test1.shape)
print(y_train2.shape)
print(y_test2.shape)
print(X_train_embed.shape)
print(X_test_embed.shape)
print(future_matches_calculado_scaled.shape)

(61353, 92)
(5789, 92)
(61353,)
(5789,)
(61353,)
(5789,)
(61353, 3)
(5789, 3)
(43, 95)


Função de perda

In [25]:
"""import tensorflow as tf

def custom_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])

    avg = tf.reduce_mean(y_true)
    abs_error = tf.abs(y_true - y_pred)
    distance_to_avg = tf.abs(y_pred - avg)
    reward = tf.math.log(distance_to_avg + 3)
    penalty = 2 * abs_error / (distance_to_avg + 3)
    
    # Aplicando a função tanh ao resultado de (penalty - reward)
    normalized_diff = tf.math.tanh(penalty - reward)
    
    # Somando com abs_error
    custom_loss_value = abs_error + normalized_diff
    
    # Garantindo que o valor mínimo da perda seja 0.0001
    custom_loss_value = tf.maximum(custom_loss_value, 0.0001)
    
    return tf.reduce_mean(custom_loss_value)"""

In [28]:
import tensorflow as tf

def custom_unified_loss(y_true, y_pred):
    # Separar y_true e y_pred para cada time
    y_true_team1, y_true_team2 = y_true[:, 0], y_true[:, 1]
    y_pred_team1, y_pred_team2 = y_pred[:, 0], y_pred[:, 1]
    
    # Calcular a média para cada time
    avg1 = tf.reduce_mean(y_true_team1)
    avg2 = tf.reduce_mean(y_true_team2)

    # Calcular o erro absoluto relativo para cada time
    normalized_error_team1 = tf.abs(y_true[:, 0] - y_pred[:, 0]) 
    normalized_error_team2 = tf.abs(y_true[:, 1] - y_pred[:, 1]) *(avg1/avg2)
    abs_error = (normalized_error_team1 + normalized_error_team2) / 2

    #_val_mae_diff = np.mean(np.abs((val_predict[:, 0] - val_predict[:, 1]) - (val_targ[:, 0] - val_targ[:, 1])))
       # ...
    dif_prev = (y_pred_team1 - y_pred_team2)
    dif_true = (y_true_team1 - y_true_team2)
    
    # Calcula a diferença absoluta entre as diferenças previstas e verdadeiras
    abs_diff = tf.abs(dif_prev - dif_true)
    
    # Calcula um termo de bônus usando a função sigmoidal
    bonus_term = tf.math.tanh(10 * (1 - abs_diff))
    
    # Se a diferença é maior ou igual a 4 (ou menor ou igual a -4), aplica um fator negativo
    condition1 = tf.logical_and(dif_true >= 4, dif_prev >= 4)
    condition2 = tf.logical_and(dif_true <= -4, dif_prev <= -4)
    condition3 = abs_diff <= 1.5

    final_condition = tf.logical_or(tf.logical_or(condition1, condition2), condition3)

    abs_diff_acc = tf.where(final_condition, -1 * (abs_diff + bonus_term), abs_diff + bonus_term)


    # Calcular a diferença entre as previsões para as médias
    distance_to_avg1 = tf.abs(y_pred_team1 - avg1)
    distance_to_avg2 = tf.abs(y_pred_team2 - avg2)
    
    # Calcular os rewards
    reward1 = tf.math.log(distance_to_avg1 + 1)
    reward2 = tf.math.log(distance_to_avg2 + 1)

    # Calcular as penalties
    penalty1 = 1 * normalized_error_team1 / (distance_to_avg1 + 1)
    penalty2 = 1 * normalized_error_team2 / (distance_to_avg2 + 1)    

    # Aplicando a função tanh ao resultado de (penalty - reward)
    normalized_diff1 = tf.math.tanh(penalty1 - reward1)
    normalized_diff2 = tf.math.tanh(penalty2 - reward2)
    normalized_diff = tf.math.tanh((normalized_diff1 + normalized_diff2)/2)

    # combined_error
    combined_error = abs_error + abs_diff_acc + normalized_diff
    
    # Garantindo que o valor mínimo da perda seja 0.0001
    combined_error_value = tf.maximum(combined_error, 0.0001)

    return tf.reduce_mean(combined_error_value)


Testes de funções de perda

In [61]:
import numpy as np
import tensorflow as tf

def custom_unified_loss2(y_true, y_pred):
    # Separar y_true e y_pred para cada time
    y_true_team1, y_true_team2 = y_true[:, 0], y_true[:, 1]
    y_pred_team1, y_pred_team2 = y_pred[:, 0], y_pred[:, 1]
    
    # Calcular a média para cada time
    avg1 = 6.6
    avg2 = 4.6

    # Calcular o erro absoluto relativo para cada time
    normalized_error_team1 = tf.abs(y_true[:, 0] - y_pred[:, 0]) 
    normalized_error_team2 = tf.abs(y_true[:, 1] - y_pred[:, 1]) *(avg1/avg2)
    abs_error = (normalized_error_team1 + normalized_error_team2) / 2

    #_val_mae_diff = np.mean(np.abs((val_predict[:, 0] - val_predict[:, 1]) - (val_targ[:, 0] - val_targ[:, 1])))
       # ...
    dif_prev = (y_pred_team1 - y_pred_team2)
    dif_true = (y_true_team1 - y_true_team2)
    
    # Calcula a diferença absoluta entre as diferenças previstas e verdadeiras
    abs_diff = tf.abs(dif_prev - dif_true)
    
    # Calcula um termo de bônus usando a função sigmoidal
    bonus_term = tf.math.tanh(10 * (1 - abs_diff))
    
    # Se a diferença é maior ou igual a 4 (ou menor ou igual a -4), aplica um fator negativo
    if (dif_true >= 4 and dif_prev >= 4) or (dif_true <= -4 and dif_prev <= -4) or (abs_diff <= 1.5):
        abs_diff_acc = -1 * (abs_diff + bonus_term)
    else:
        abs_diff_acc = abs_diff + bonus_term


    # Calcular a diferença entre as previsões para as médias
    distance_to_avg1 = tf.abs(y_pred_team1 - avg1)
    distance_to_avg2 = tf.abs(y_pred_team2 - avg2)
    
    # Calcular os rewards
    reward1 = tf.math.log(distance_to_avg1 + 1)
    reward2 = tf.math.log(distance_to_avg2 + 1)

    # Calcular as penalties
    penalty1 = 1 * normalized_error_team1 / (distance_to_avg1 + 1)
    penalty2 = 1 * normalized_error_team2 / (distance_to_avg2 + 1)    

    # Aplicando a função tanh ao resultado de (penalty - reward)
    normalized_diff1 = tf.math.tanh(penalty1 - reward1)
    normalized_diff2 = tf.math.tanh(penalty2 - reward2)
    normalized_diff = tf.math.tanh((normalized_diff1 + normalized_diff2)/2)

    # Print rewards, penalties, and normalized_diff
    print(f"reward1: {reward1.numpy()}, penalty1: {penalty1.numpy()}, normalized_diff1: {normalized_diff1.numpy()}")
    print(f"reward2: {reward2.numpy()}, penalty2: {penalty2.numpy()}, normalized_diff2: {normalized_diff2.numpy()}")
    print(f"bonus_term {bonus_term.numpy()}--> abs_diff_acc: {abs_diff_acc.numpy()}, abs_error: {abs_error.numpy()}, normalized_diff: {normalized_diff.numpy()}")  # Novo valor impresso


    # combined_error
    combined_error = abs_error + abs_diff_acc + normalized_diff
    
    # Garantindo que o valor mínimo da perda seja 0.0001
    combined_error_value = tf.maximum(combined_error, 0.0001)

    return tf.reduce_mean(combined_error_value)

# Loop para gerar valores aleatórios e calcular a perda
for _ in range(20):
    # Gerando valores aleatórios para y_true e y_pred
    y_true_sample = np.random.randint(0, 16, size=(1, 2))
    y_pred_sample = np.random.randint(0, 16, size=(1, 2))
    
    # Convertendo para tensores do TensorFlow
    y_true_tensor = tf.convert_to_tensor(y_true_sample, dtype=tf.float32)
    y_pred_tensor = tf.convert_to_tensor(y_pred_sample, dtype=tf.float32)
    
    # Calculando a perda
    loss_value = custom_unified_loss2(y_true_tensor, y_pred_tensor).numpy()
    
    # Imprimindo os resultados
    print(f"y_true_team1: {y_true_sample[0][0]}, y_pred_team1: {y_pred_sample[0][0]}, y_true_team2: {y_true_sample[0][1]}, y_pred_team2: {y_pred_sample[0][1]}, Loss: {loss_value:.4f}\n")


reward1: [2.2407095], penalty1: [0.10638298], normalized_diff1: [-0.9723854]
reward2: [2.00148], penalty2: [0.3877791], normalized_diff2: [-0.9237051]
bonus_term [-1.]--> abs_diff_acc: [2.], abs_error: [1.9347826], normalized_diff: [-0.73889685]
y_true_team1: 14, y_pred_team1: 15, y_true_team2: 13, y_pred_team2: 11, Loss: 3.1959

reward1: [1.686399], penalty1: [0.18518518], normalized_diff1: [-0.9053674]
reward2: [0.9555114], penalty2: [4.9665556], normalized_diff2: [0.9993438]
bonus_term [-1.]--> abs_diff_acc: [7.], abs_error: [6.956522], normalized_diff: [0.04695366]
y_true_team1: 12, y_pred_team1: 11, y_true_team2: 12, y_pred_team2: 3, Loss: 14.0035

reward1: [0.47000358], penalty1: [2.5000002], normalized_diff1: [0.9660868]
reward2: [1.856298], penalty2: [0.22418478], normalized_diff2: [-0.92636186]
bonus_term [-1.]--> abs_diff_acc: [2.], abs_error: [2.7173913], normalized_diff: [0.01985986]
y_true_team1: 10, y_pred_team1: 6, y_true_team2: 11, y_pred_team2: 10, Loss: 4.7373

reward

Concatenando ambos Ys em um único tensor

In [29]:
import numpy as np

# Suponha que y_train1 e y_train2 são seus vetores de saída originais
# Eles têm o shape (comprimento,)

# Calcular a diferença entre y_train1 e y_train2
y_train_diff = y_train1 - y_train2
y_test_diff = y_test1 - y_test2
# Concatenar ao longo de uma nova dimensão para criar um tensor de saída com shape (comprimento, 3)
y_train_combined = np.stack((y_train1, y_train2, y_train_diff), axis=-1)
y_test_combined = np.stack((y_test1, y_test2, y_test_diff), axis=-1)


print(y_train_combined.shape)
print(y_test_combined.shape)


(61353, 3)
(5789, 3)


Arquitetura da NN

In [30]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam

# ==== Parte 1: Definindo os Inputs para o Embedding ====
# Estes são os inputs que vão alimentar os embeddings. 
# Cada input tem a dimensão de (1,) porque cada jogo tem exatamente um 'team1', um 'team2', e um 'championship'.
team1_input = Input(shape=(1,), name='Team1-Input')
team2_input = Input(shape=(1,), name='Team2-Input')
champ_input = Input(shape=(1,), name='Championship-Input')

# ==== Parte 2: Criando os Embeddings ====
# n_teams e n_champ são o número de times e campeonatos únicos, respectivamente.
# O output_dim é um hiperparâmetro para você ajustar. Ele define o tamanho do espaço de embedding.

# Embedding para o time 1
team1_embedding = Embedding(input_dim=n_teams, output_dim=50, name='Team1-Embedding')(team1_input)  # output_dim ajustável

# Embedding para o time 2
team2_embedding = Embedding(input_dim=n_teams, output_dim=50, name='Team2-Embedding')(team2_input)  # output_dim ajustável

# Embedding para o campeonato
champ_embedding = Embedding(input_dim=n_champ, output_dim=5, name='Championship-Embedding')(champ_input)  # output_dim ajustável

# ==== Parte 3: Achatando os Embeddings ====
# Cada embedding precisa ser achatado para ser concatenado posteriormente
team1_embedding = Flatten()(team1_embedding)
team2_embedding = Flatten()(team2_embedding)
champ_embedding = Flatten()(champ_embedding)

# ==== Parte 4: Outras Características ====
# Este é o input para as outras características (já escaladas) do seu conjunto de dados.
other_features_input = Input(shape=(X_train_scaled.shape[1],), name='Other-Features-Input')

# ==== Parte 5: Concatenando Tudo ====
# Aqui, todos os embeddings e as outras características são concatenados em um único vetor
merged = Concatenate()([team1_embedding, team2_embedding, champ_embedding, other_features_input])

# ==== Parte 6: Camadas Ocultas ====
# Estes são os neurônios e camadas totalmente conectadas (Dense) onde a "aprendizagem" real acontece.
# Você pode ajustar o número de neurônios, a função de ativação, e outros hiperparâmetros aqui.
from tensorflow.keras.regularizers import l1, l2

hidden_layer = Dense(1024, activation='relu', kernel_regularizer=l2(0.0001))(merged)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(512, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(256, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(128, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(64, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(32, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

#hidden_layer = Dense(8, activation='tanh')(hidden_layer)
#hidden_layer = Dropout(0.2)(hidden_layer)

# ==== Parte 7: Camada de Saída ====
# Esta é a camada de saída. A função de ativação 'linear' é usada para regressão.
output = Dense(3, activation='linear', name='Output-Layer')(hidden_layer)

# ==== Parte 8: Compilando o Modelo ====
# Finalmente, o modelo é compilado. O otimizador Adam é usado, com uma taxa de aprendizagem de 0.001.
# A perda é definida como 'mean_squared_error', que é comum para problemas de regressão.
model = Model(
    inputs=[team1_input, team2_input, champ_input, other_features_input], 
    outputs=[output]
)


In [35]:
from tensorflow.keras.optimizers import Adam
import numpy as np

# Parâmetros para Early Stopping
patience = 10
best_val_mae_diff = np.inf
epochs_without_improvement = 0

# Número de épocas
n_epochs = 100

# Compilando o modelo unificado
model.compile(optimizer=Adam(0.001), loss=custom_unified_loss, metrics=['mean_absolute_error'])

# Loop de treinamento
for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    
    # Treinamento para uma época
    history = model.fit(
        [X_train_embed['team1'], X_train_embed['team2'], X_train_embed['championship'], X_train_scaled], 
        y_train_combined, 
        epochs=1,
        batch_size=128,
        verbose=1,
        validation_data=([X_test_embed['team1'], X_test_embed['team2'], X_test_embed['championship'], X_test_scaled], y_test_combined)
    )
    
    # Calculando métricas adicionais
    val_predict = model.predict([X_test_embed['team1'], X_test_embed['team2'], X_test_embed['championship'], X_test_scaled])
    val_targ = y_test_combined

    _val_mae_team1 = np.mean(np.abs(val_predict[:, 0] - val_targ[:, 0]))
    _val_mae_team2 = np.mean(np.abs(val_predict[:, 1] - val_targ[:, 1]))
    _val_mae_diff = np.mean(np.abs((val_predict[:, 0] - val_predict[:, 1]) - (val_targ[:, 0] - val_targ[:, 1])))
    _val_mae_diff_output = np.mean(np.abs(val_predict[:, 2] - val_targ[:, 2]))  # Nova métrica para a terceira saída

    print(f" - val_mae_team1: {_val_mae_team1:.4f} - val_mae_team2: {_val_mae_team2:.4f} - val_mae_diff: {_val_mae_diff:.4f} - val_mae_diff_output: {_val_mae_diff_output:.4f}")

    # Lógica de Early Stopping
    if _val_mae_diff < best_val_mae_diff:
        best_val_mae_diff = _val_mae_diff
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("Early stopping triggered.")
            break


Epoch 1/100
181/181 [==============================] - 1s 2ms/step
 - val_mae_team1: 2.2739 - val_mae_team2: 2.0301 - val_mae_diff: 3.2330 - val_mae_diff_output: 3.5171
Epoch 2/100
181/181 [==============================] - 0s 2ms/step
 - val_mae_team1: 2.2591 - val_mae_team2: 2.0070 - val_mae_diff: 3.2337 - val_mae_diff_output: 3.3802
Epoch 3/100
181/181 [==============================] - 0s 2ms/step
 - val_mae_team1: 2.3199 - val_mae_team2: 2.0154 - val_mae_diff: 3.2501 - val_mae_diff_output: 3.4021
Epoch 4/100
181/181 [==============================] - 0s 2ms/step
 - val_mae_team1: 2.2542 - val_mae_team2: 1.9953 - val_mae_diff: 3.2498 - val_mae_diff_output: 3.4499
Epoch 5/100
181/181 [==============================] - 0s 2ms/step
 - val_mae_team1: 2.2560 - val_mae_team2: 2.0120 - val_mae_diff: 3.2518 - val_mae_diff_output: 3.4603
Epoch 6/100
181/181 [==============================] - 0s 2ms/step
 - val_mae_team1: 2.2704 - val_mae_team2: 2.0366 - val_mae_diff: 3.2405 - val_mae_diff_o

In [ ]:
#2.1901447772979736
#1.996545672416687

In [36]:
future_matches_calculado_scaled.shape

(43, 95)

In [37]:
future_matches_calculado_scaled.head()

,team1,team2,team1_goals,team2_goals,championship,team1_big_wins_last5,team1_big_losses_last5,team2_big_wins_last5,team2_big_losses_last5,team1_ah-2.5_wins_last5,...,team2_winning_streak,team2_undefeated_streak,team2_losing_streak,team2_without_winning_streak,avg_points_lasts5_2,team2_strength,championship_points_2,rested_4_days_or_more_2,team1_suspended_players,team2_suspended_players
127,533,519,-1.192433,-1.036144,1,-0.907378,1.233839,-0.939662,1.319616,1.526092,...,-0.569612,-0.360440,-0.525751,1.187144,-1.825696,-3.420264,-3.846691,0.443706,-0.334677,-0.305174
1031,515,499,-1.192433,-1.036144,21,1.257310,0.123596,0.117516,0.184672,-0.333442,...,0.986754,-0.059744,-0.525751,-0.690139,0.629454,2.914147,3.052626,0.443706,-0.334677,-0.305174
372,471,472,-1.192433,-1.036144,2,0.174966,-0.986648,1.174694,-0.950273,-0.333442,...,0.208571,-0.360440,-0.525751,-0.690139,0.629454,0.934421,0.217290,0.443706,2.587813,-0.305174
373,508,517,-1.192433,-1.036144,2,-0.907378,-0.986648,-0.939662,0.184672,-0.333442,...,-0.569612,-0.360440,-0.525751,2.688970,-1.518802,-1.020477,-0.570303,0.443706,-0.334677,-0.305174
374,479,502,-1.192433,-1.036144,2,2.339654,-0.986648,-0.939662,0.184672,0.596325,...,-0.569612,0.240951,-0.525751,-0.314683,0.322560,0.875556,1.162402,0.443706,-0.334677,-0.305174


Output!!!!!

In [38]:
# Inicializando um dicionário vazio para armazenar as previsões
corner_predictions = {}

# Iterar sobre cada linha em 'future_matches' e 'future_matches_calculado_scaled'
for (index1, row1), (index2, row2) in zip(future_matches.iterrows(), future_matches_calculado_scaled.iterrows()):
    # Prepare os dados de entrada para a previsão
    team1_input_data = np.array([[row2['team1']]], dtype=np.float32)
    team2_input_data = np.array([[row2['team2']]], dtype=np.float32)
    champ_input_data = np.array([[row2['championship']]], dtype=np.float32)
    
    # Certifique-se de que 'other_features_data' contém todas as outras características na mesma ordem que foram usadas para treinar o modelo
    other_features_data = np.array([row2.drop(['team1', 'team2', 'championship']).astype(np.float32)])
    
    # Faça a previsão usando o modelo unificado
    pred = model.predict([team1_input_data, team2_input_data, champ_input_data, other_features_data])
    
    # As previsões para ambos os times estão na mesma saída, então vamos separá-las
    pred_team1, pred_team2 = pred[0][0], pred[0][1]
    
    # Armazenar as previsões no dicionário
    corner_predictions[row1['team1']] = pred_team1
    corner_predictions[row1['team2']] = pred_team2
    
    # Imprimir as previsões
    print(f"Prediction for {row1['team1']}: {pred_team1} corners")
    print(f"Prediction for {row1['team2']}: {pred_team2} corners")

# Exibindo o dicionário de previsões
print("Corner Predictions:", corner_predictions)


1/1 [==============================] - 0s 134ms/step
Prediction for oud-heverlee leuven: 6.986445903778076 corners
Prediction for kv kortrijk: 3.306103467941284 corners
1/1 [==============================] - 0s 26ms/step
Prediction for ifk varnamo: 4.0239129066467285 corners
Prediction for elfsborg: 5.817728042602539 corners
1/1 [==============================] - 0s 25ms/step
Prediction for athletico-pr: 4.1945929527282715 corners
Prediction for atlético mineiro: 3.209925413131714 corners
1/1 [==============================] - 0s 25ms/step
Prediction for goiás: 5.023271560668945 corners
Prediction for internacional: 2.977043390274048 corners
1/1 [==============================] - 0s 23ms/step
Prediction for botafogo: 6.98805570602417 corners
Prediction for flamengo: 3.306223154067993 corners
1/1 [==============================] - 0s 24ms/step
Prediction for halmstad: 4.126588344573975 corners
Prediction for ik sirius: 3.4060025215148926 corners
1/1 [==============================] - 0s

In [39]:
# Transformar o dicionário em uma lista de listas, quebrando a cada 2 itens
items = list(corner_predictions.items())
rows = [items[i:i + 2] for i in range(0, len(items), 2)]

# Criar um DataFrame a partir da lista de listas
df = pd.DataFrame(rows, columns=['Team1', 'Team2'])

# Separar as tuplas em duas colunas separadas para os nomes das equipes e as previsões
df[['Team1_name', 'Team1_prediction']] = pd.DataFrame(df['Team1'].tolist(), index=df.index)
df[['Team2_name', 'Team2_prediction']] = pd.DataFrame(df['Team2'].tolist(), index=df.index)

# Descartar as colunas originais e reordenar
df = df[['Team1_name', 'Team1_prediction', 'Team2_name', 'Team2_prediction']]


df

,Team1_name,Team1_prediction,Team2_name,Team2_prediction
0,oud-heverlee leuven,6.986446,kv kortrijk,3.306103
1,ifk varnamo,4.023913,elfsborg,5.817728
2,athletico-pr,4.194593,atlético mineiro,3.209925
3,goiás,5.023272,internacional,2.977043
4,botafogo,6.988056,flamengo,3.306223
5,halmstad,4.126588,ik sirius,3.406003
6,kv mechelen,6.894626,eupen,3.332961
7,cercle brugge,6.986990,kvc westerlo,3.302955
8,américa-mg,4.579938,santos,2.843539
9,cruzeiro,4.380726,red bull bragantino,4.755130


In [40]:
df.shape

(43, 4)

In [41]:
#STEP 2

from scipy.stats import poisson
import numpy as np
import pandas as pd

def poisson_probabilities(lam, max_corners=15):
    probs = [poisson.pmf(k, lam) for k in range(max_corners + 1)]
    probs.append(1 - sum(probs))
    return probs

def calculate_probabilities(team1_corners_prediction, team2_corners_prediction):
    team1_corners_probabilities = poisson_probabilities(team1_corners_prediction)
    team2_corners_probabilities = poisson_probabilities(team2_corners_prediction)

    joint_prob_matrix = np.outer(team1_corners_probabilities, team2_corners_probabilities)

    team1_win_prob = np.sum(np.tril(joint_prob_matrix, -1))
    draw_prob = np.sum(np.diag(joint_prob_matrix))
    team2_win_prob = np.sum(np.triu(joint_prob_matrix, 1))

    team1_minus35_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=4])
    team1_plus35_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=-3])
    team2_minus35_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=4])
    team2_plus35_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=-3])

    team1_minus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=3])
    team1_plus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=-2])
    team2_minus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=3])
    team2_plus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=-2])

    team1_minus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=2])
    team1_plus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=-1])
    team2_minus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=2])
    team2_plus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=-1])

    team1_over45_prob = 1 - sum(team1_corners_probabilities[:5])
    team1_under45_prob = sum(team1_corners_probabilities[:5])
    team2_over45_prob = 1 - sum(team2_corners_probabilities[:5])
    team2_under45_prob = sum(team2_corners_probabilities[:5])

    team1_over55_prob = 1 - sum(team1_corners_probabilities[:6])
    team1_under55_prob = sum(team1_corners_probabilities[:6])
    team2_over55_prob = 1 - sum(team2_corners_probabilities[:6])
    team2_under55_prob = sum(team2_corners_probabilities[:6])

    team1_over65_prob = 1 - sum(team1_corners_probabilities[:7])
    team1_under65_prob = sum(team1_corners_probabilities[:7])
    team2_over65_prob = 1 - sum(team2_corners_probabilities[:7])
    team2_under65_prob = sum(team2_corners_probabilities[:7])
    
    return (team1_win_prob, draw_prob, team2_win_prob,
        team1_minus15_prob, team1_plus15_prob, team2_minus15_prob, team2_plus15_prob,
        team1_minus25_prob, team1_plus25_prob, team2_minus25_prob, team2_plus25_prob,
        team1_minus35_prob, team1_plus35_prob, team2_minus35_prob, team2_plus35_prob,
        team1_over45_prob, team1_under45_prob, team2_over45_prob, team2_under45_prob,
        team1_over55_prob, team1_under55_prob, team2_over55_prob, team2_under55_prob,
        team1_over65_prob, team1_under65_prob, team2_over65_prob, team2_under65_prob)





In [42]:
# Inicializando o DataFrame para armazenar as odds
odds_df = pd.DataFrame(columns=['Team 1', 'Team 2', 'Team 1 Win Odd', 'Draw Odd', 'Team 2 Win Odd',
                                'Team 1 -1.5 Odd', 'Team 1 +1.5 Odd', 'Team 2 -1.5 Odd', 'Team 2 +1.5 Odd',
                                'Team 1 -2.5 Odd', 'Team 1 +2.5 Odd', 'Team 2 -2.5 Odd', 'Team 2 +2.5 Odd',
                                'Team 1 -3.5 Odd', 'Team 1 +3.5 Odd', 'Team 2 -3.5 Odd', 'Team 2 +3.5 Odd',
                                'Team 1 Over 4.5', 'Team 1 Under 4.5', 'Team 2 Over 4.5', 'Team 2 Under 4.5',
                                'Team 1 Over 5.5', 'Team 1 Under 5.5', 'Team 2 Over 5.5', 'Team 2 Under 5.5',
                                'Team 1 Over 6.5', 'Team 1 Under 6.5', 'Team 2 Over 6.5', 'Team 2 Under 6.5'])

# Iterar sobre cada linha no DataFrame 'df'
for index, row in df.iterrows():
    team1_name = row['Team1_name']
    team1_prediction = row['Team1_prediction']
    team2_name = row['Team2_name']
    team2_prediction = row['Team2_prediction']
    
    # Calculando as probabilidades usando a função do STEP 2
    probabilities = calculate_probabilities(team1_prediction, team2_prediction)
    
    # Calculando as odds
    odds = [1 / prob for prob in probabilities]
    
    # Adicionando as odds ao DataFrame
    odds_df.loc[len(odds_df)] = [team1_name, team2_name] + odds

    # Inicializando as novas colunas
odds_df['date'] = None
odds_df['championship'] = None

# Preenchendo as novas colunas
for index, row in odds_df.iterrows():
    team1 = row['Team 1']
    team2 = row['Team 2']
    
    # Encontrando a linha correspondente em 'future_matches'
    match_row = future_matches[(future_matches['team1'] == team1) & (future_matches['team2'] == team2)]
    
    if not match_row.empty:
        # Se encontrarmos uma linha correspondente, atualizamos 'odds_df'
        odds_df.at[index, 'date'] = match_row.iloc[0]['date']
        odds_df.at[index, 'championship'] = match_row.iloc[0]['championship']

# Converte a coluna 'date' para o tipo de data do pandas
odds_df['date'] = pd.to_datetime(odds_df['date'])

# Formata a coluna 'date' para o formato de data brasileiro (dd/mm/yyyy)
odds_df['date'] = odds_df['date'].dt.strftime('%d/%m/%Y')

odds_df = odds_df.sort_values(['championship', 'date'])

# Lista das colunas atuais
cols = odds_df.columns.tolist()

# Removendo 'date' e 'championship' da lista
cols.remove('date')
cols.remove('championship')

# Reordenando as colunas
new_cols = ['date', 'championship'] + cols

# Atualizando o DataFrame com a nova ordem de colunas
odds_df = odds_df[new_cols]

odds_df.to_excel("output_corners_NNNN_AH.xlsx", index=False)

# Exibindo o DataFrame de odds
print(odds_df.shape)


(43, 31)
